In [1]:
import pandas as pd

In [2]:
# Read in csv
data = pd.read_csv('Netflix_Userbase.csv')

data

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month
...,...,...,...,...,...,...,...,...,...,...
2495,2496,Premium,14,25-07-22,12-07-23,Spain,28,Female,Smart TV,1 Month
2496,2497,Basic,15,04-08-22,14-07-23,Spain,33,Female,Smart TV,1 Month
2497,2498,Standard,12,09-08-22,15-07-23,United States,38,Male,Laptop,1 Month
2498,2499,Standard,13,12-08-22,12-07-23,Canada,48,Female,Tablet,1 Month


In [3]:
# Set User ID as index
data = data.set_index('User ID')

# Drop 'Plan Duration' series since the whole column had the same value anyways
data = data.drop(columns=['Plan Duration'])

# Convert date columns to datetime datatypes
data['Join Date'] = pd.to_datetime(data['Join Date'])
data['Last Payment Date'] = pd.to_datetime(data['Last Payment Date'])

#Check data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 1 to 2500
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Subscription Type  2500 non-null   object        
 1   Monthly Revenue    2500 non-null   int64         
 2   Join Date          2500 non-null   datetime64[ns]
 3   Last Payment Date  2500 non-null   datetime64[ns]
 4   Country            2500 non-null   object        
 5   Age                2500 non-null   int64         
 6   Gender             2500 non-null   object        
 7   Device             2500 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 175.8+ KB


In [4]:
# View data
data

,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device
User ID,,,,,,,,
1,Basic,10,2022-01-15,2023-10-06,United States,28,Male,Smartphone
2,Premium,15,2021-05-09,2023-06-22,Canada,35,Female,Tablet
3,Standard,12,2023-02-28,2023-06-27,United Kingdom,42,Male,Smart TV
4,Standard,12,2022-10-07,2023-06-26,Australia,51,Female,Laptop
5,Basic,10,2023-01-05,2023-06-28,Germany,33,Male,Smartphone
...,...,...,...,...,...,...,...,...
2496,Premium,14,2022-07-25,2023-12-07,Spain,28,Female,Smart TV
2497,Basic,15,2022-04-08,2023-07-14,Spain,33,Female,Smart TV
2498,Standard,12,2022-09-08,2023-07-15,United States,38,Male,Laptop


In [5]:
# Isolate unique countries with counts
countries = data.value_counts('Country')

# Convert that value_counts to a df, renaming column appropriately
countries = countries.to_frame()
countries = countries.rename(columns={0: "Count"})

# Give df a fresh index
countries = countries.reset_index()

# Generate country code for each country
countries['country_id'] = range(len(countries))
countries['country_id'] = [f'c{i}' for i in range(len(countries))]

# Reorder columns
countries = countries[['country_id','Country','Count']]

countries


,country_id,Country,Count
0,c0,Spain,451
1,c1,United States,451
2,c2,Canada,317
3,c3,Australia,183
4,c4,Brazil,183
5,c5,France,183
6,c6,Germany,183
7,c7,Italy,183
8,c8,Mexico,183
9,c9,United Kingdom,183


In [6]:
# Merge country codes into original dataset
data = data.merge(countries, left_on='Country', right_on='Country')
data


,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,country_id,Count
0,Basic,10,2022-01-15,2023-10-06,United States,28,Male,Smartphone,c1,451
1,Basic,10,2022-05-16,2023-06-22,United States,31,Female,Smartphone,c1,451
2,Premium,15,2023-10-06,2023-06-22,United States,26,Female,Laptop,c1,451
3,Premium,15,2023-03-02,2023-06-22,United States,28,Female,Smart TV,c1,451
4,Premium,15,2023-01-20,2023-06-22,United States,31,Female,Laptop,c1,451
...,...,...,...,...,...,...,...,...,...,...
2495,Basic,14,2022-06-09,2023-07-13,Italy,39,Male,Smartphone,c7,183
2496,Basic,13,2022-10-29,2023-07-13,Italy,47,Female,Smartphone,c7,183
2497,Basic,15,2022-03-11,2023-11-07,Italy,30,Male,Tablet,c7,183
2498,Basic,13,2022-07-08,2023-07-14,Italy,47,Male,Laptop,c7,183


In [7]:
# Identify columns, drop country count column
cols = ['Subscription Type', 'Age', 'Gender', 'Device', 'Monthly Revenue', 'Join Date',
       'Last Payment Date', 'Country', 'country_id']

# Set data w/ new columns
data = data[cols]

data


,Subscription Type,Age,Gender,Device,Monthly Revenue,Join Date,Last Payment Date,Country,country_id
0,Basic,28,Male,Smartphone,10,2022-01-15,2023-10-06,United States,c1
1,Basic,31,Female,Smartphone,10,2022-05-16,2023-06-22,United States,c1
2,Premium,26,Female,Laptop,15,2023-10-06,2023-06-22,United States,c1
3,Premium,28,Female,Smart TV,15,2023-03-02,2023-06-22,United States,c1
4,Premium,31,Female,Laptop,15,2023-01-20,2023-06-22,United States,c1
...,...,...,...,...,...,...,...,...,...
2495,Basic,39,Male,Smartphone,14,2022-06-09,2023-07-13,Italy,c7
2496,Basic,47,Female,Smartphone,13,2022-10-29,2023-07-13,Italy,c7
2497,Basic,30,Male,Tablet,15,2022-03-11,2023-11-07,Italy,c7
2498,Basic,47,Male,Laptop,13,2022-07-08,2023-07-14,Italy,c7


In [30]:
# Export cleaned user data to csv and json
data.to_csv('clean_user_data.csv')
data.to_json('clean_user_data.json')

# Export countries data to csv
countries.to_csv('country_data.csv')
countries.to_json('country_data.json')


In [31]:
# This is what Bear's query is going to look like
bears_data = data[['Gender','Country']]
bears_data.to_json('bears_data.json')
bears_data


,Gender,Country
0,Male,United States
1,Female,United States
2,Female,United States
3,Female,United States
4,Female,United States
...,...,...
2495,Male,Italy
2496,Female,Italy
2497,Male,Italy
2498,Male,Italy
